<a href="https://colab.research.google.com/github/AntonioCRCosta/antonio-c-r-costa/blob/main/minha_parte_no_desafio_rascunho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Bibliotecas
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib as mt
from scipy.optimize import minimize

In [11]:

# Baixando dados dos índices das empresas e commodities
jbs = yf.download('JBSS3.SA', start='2010-11-19', end='2023-11-19')['Close']
marfig = yf.download('MRFG3.SA', start='2010-11-19', end='2023-11-19')['Close']
minupar = yf.download('MNPR3.SA', start='2010-11-19', end='2023-11-19')['Close']
minerva = yf.download('BEEF3.SA', start='2010-11-19', end='2023-11-19')['Close']
excelsior = yf.download('BAUH4.SA', start='2010-11-19', end='2023-11-19')['Close']
brf = yf.download('BRFS3.SA', start='2010-11-19', end='2023-11-19')['Close']
gado = yf.download('LE=F', start='2010-11-19', end='2023-11-19')['Close']
porco = yf.download('HE=F', start='2010-11-19', end='2023-11-19')['Close']
ibov = yf.download('^BVSP', start='2010-11-19', end='2023-11-19')['Close']
nasdaq = yf.download('NQ=F', start='2010-11-19', end='2023-11-19')['Close']

# Concatenando todas as séries em um DataFrame
dados_empresas = pd.concat([jbs, marfig, minupar, minerva, excelsior, brf, gado, porco, ibov,nasdaq], axis=1)
dados_empresas.columns = ['jbs', 'marfig', 'minupar', 'minerva', 'excelsior', 'brf', 'gado', 'porco', 'ibov','nasdaq']
# Definindo a frequência como diária (todos os dias)
dados_empresas = dados_empresas.asfreq('D', method='pad')

dados_empresas


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,jbs,marfig,minupar,minerva,excelsior,brf,gado,porco,ibov,nasdaq
Date,,,,,,,,,,
2010-11-19 00:00:00+00:00,6.310000,13.383684,35.00,6.401249,2.750000,23.578686,101.449997,69.125000,70898.0,2133.50
2010-11-20 00:00:00+00:00,6.310000,13.383684,35.00,6.401249,2.750000,23.578686,101.449997,69.125000,70898.0,2133.50
2010-11-21 00:00:00+00:00,6.310000,13.383684,35.00,6.401249,2.750000,23.578686,101.449997,69.125000,70898.0,2133.50
2010-11-22 00:00:00+00:00,6.250000,13.473174,35.00,6.411082,2.900000,23.226480,100.925003,69.400002,69633.0,2154.50
2010-11-23 00:00:00+00:00,6.060000,13.294195,35.00,6.194757,2.710000,22.750528,101.550003,69.824997,67953.0,2119.00
...,...,...,...,...,...,...,...,...,...,...
2023-11-13 00:00:00+00:00,21.000000,7.560000,17.42,6.880000,71.889999,12.310000,174.925003,73.349998,120376.0,15547.00
2023-11-14 00:00:00+00:00,20.879999,7.900000,17.67,7.040000,71.889999,12.830000,175.850006,72.300003,123328.0,15879.75
2023-11-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,177.774994,71.050003,NaN,15889.25


In [21]:
# Função para preencher NaN com a média da última e próxima observação não NaN
def preencher_media_vizinhos(col):
    """
    Preenche os valores NaN de uma coluna com a média dos valores anterior e próximo não NaN.

    Parâmetros:
    col (pd.Series): Série com valores que podem conter NaNs.

    Retorna:
    pd.Series: Série com valores NaN preenchidos.
    """
    col = col.copy()  # Cópia da coluna para evitar alterações diretas
    for i in range(len(col)):
        if pd.isna(col[i]):
            prev_val, next_val = None, None

            # Procurando o último valor não NaN
            j = i - 1
            while j >= 0 and pd.isna(col[j]):
                j -= 1
            if j >= 0:
                prev_val = col[j]

            # Procurando o próximo valor não NaN
            k = i + 1
            while k < len(col) and pd.isna(col[k]):
                k += 1
            if k < len(col):
                next_val = col[k]

            # Preenchendo a média ou substituindo pelo valor anterior ou próximo se existir
            if prev_val is not None and next_val is not None:
                col[i] = (prev_val + next_val) / 2
            elif prev_val is not None:
                col[i] = prev_val
            elif next_val is not None:
                col[i] = next_val
    return col
# Aplicando a função em cada coluna do DataFrame
dados_empresas = dados_empresas.apply(preencher_media_vizinhos)
dados_empresas

<ipython-input-21-226f55e91c9a>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(col[i]):


,jbs,marfig,minupar,minerva,excelsior,brf,gado,porco,ibov,nasdaq
Date,,,,,,,,,,
2010-11-19 00:00:00+00:00,6.310000,13.383684,35.000,6.401249,2.750000,23.578686,101.449997,69.125000,70898.0,2133.50
2010-11-20 00:00:00+00:00,6.310000,13.383684,35.000,6.401249,2.750000,23.578686,101.449997,69.125000,70898.0,2133.50
2010-11-21 00:00:00+00:00,6.310000,13.383684,35.000,6.401249,2.750000,23.578686,101.449997,69.125000,70898.0,2133.50
2010-11-22 00:00:00+00:00,6.250000,13.473174,35.000,6.411082,2.900000,23.226480,100.925003,69.400002,69633.0,2154.50
2010-11-23 00:00:00+00:00,6.060000,13.294195,35.000,6.194757,2.710000,22.750528,101.550003,69.824997,67953.0,2119.00
...,...,...,...,...,...,...,...,...,...,...
2023-11-13 00:00:00+00:00,21.000000,7.560000,17.420,6.880000,71.889999,12.310000,174.925003,73.349998,120376.0,15547.00
2023-11-14 00:00:00+00:00,20.879999,7.900000,17.670,7.040000,71.889999,12.830000,175.850006,72.300003,123328.0,15879.75
2023-11-15 00:00:00+00:00,21.270000,8.145000,17.825,7.125000,71.889999,13.345000,177.774994,71.050003,123952.0,15889.25


In [22]:
# Calculando a variação percentual(RETORNO) entre períodos consecutivos
dados_empresas_retorno = dados_empresas.pct_change()
#tanto faz os dados de 2010 n serem possíveis...

In [23]:
#Matriz covariância dos retornos dos ativos selecionados (em pares para saber como o retorno de um ativo se move com relação ao outro)...
#...para usar no markowitz usando ExponentialMovingWindow (ewm)

# Definindo alpha para o decaimento exponencial
alpha = 0.2  #86% do peso da covariância está nas 20 primerias amostras

# Calculando a matriz de covariância ponderada exponencialmente entre as séries temporais
matriz_covariancia_exponencial = dados_empresas_retorno.ewm(alpha=0.2).cov(pairwise=True)

# Exibindo a matriz de covariância exponencial ponderada
matriz_covariancia_exponencial

jbs    marfig   minupar   minerva  \
Date                                                                          
2010-11-19 00:00:00+00:00 jbs             NaN       NaN       NaN       NaN   
                          marfig          NaN       NaN       NaN       NaN   
                          minupar         NaN       NaN       NaN       NaN   
                          minerva         NaN       NaN       NaN       NaN   
                          excelsior       NaN       NaN       NaN       NaN   
...                                       ...       ...       ...       ...   
2023-11-17 00:00:00+00:00 brf        0.000174  0.000465  0.000187  0.000310   
                          gado      -0.000026 -0.000076 -0.000057  0.000117   
                          porco     -0.000018 -0.000027 -0.000094 -0.000003   
                          ibov       0.000009  0.000116  0.000095  0.000088   
                          nasdaq     0.000006  0.000110  0.000131  0.000110   

                                        excelsior       brf      gado  \
Date                                                                    
2010-11-19 00:00:00+00:00 jbs                 NaN       NaN       NaN   
                          marfig              NaN       NaN       NaN   
                          minupar             NaN       NaN       NaN   
                          minerva             NaN       NaN       NaN   
                          excelsior           NaN       NaN       NaN   
...                                           ...       ...       ...   
2023-11-17 00:00:00+00:00 brf       -2.461369e-06  0.000951 -0.000027   
                          gado       3.659197e-08 -0.000027  0.000122   
                          porco      1.509375e-06 -0.000001 -0.000060   
                          ibov       1.092770e-06  0.000102  0.000013   
                          nasdaq     1.575108e-06  0.000075  0.000018   

                                        porco      ibov    nasdaq  
Date                                                               
2010-11-19 00:00:00+00:00 jbs             NaN       NaN       NaN  
                          marfig          NaN       NaN       NaN  
                          minupar         NaN       NaN       NaN  
                          minerva         NaN       NaN       NaN  
                          excelsior       NaN       NaN       NaN  
...                                       ...       ...       ...  
2023-11-17 00:00:00+00:00 brf       -0.000001  0.000102  0.000075  
                          gado      -0.000060  0.000013  0.000018  
                          porco      0.000163 -0.000038 -0.000024  
                          ibov      -0.000038  0.000063  0.000062  
                          nasdaq    -0.000024  0.000062  0.000078  

[47470 rows x 10 columns]

In [25]:
#MARKOWITZ
#supondo que 'dados_empresas_retorno' seja o DataFrame com os retornos dos ativos selecionados pelo modelo ADL. Vou, de maneira grosseira
#calcular a média desses retornos e considerá-lo aq em markowitz
retorno_esperado = dados_empresas_retorno.mean()
# Selecionando a última matriz de covariância calculada para usar na otimização (só não botei lá assim, pq no backtest talvez utilize outras matrizes de lá)
ultima_matriz_covariancia = matriz_covariancia_exponencial.iloc[-len(dados_empresas_retorno.columns):]
# Definindo uma taxa de retorno livre de risco para cálculo do Índice de Sharpe
taxa_livre_risco = 0.0005  # Ajustável com base na taxa anual ou diária

# Função para calcular o Índice de Sharpe negativo (para maximização)
def indice_sharpe_negativo(pesos, retornos, covariancia, taxa_livre_risco):
    retorno_portfolio = np.dot(pesos, retornos)
    risco_portfolio = np.sqrt(np.dot(pesos.T, np.dot(covariancia, pesos)))
    indice_sharpe = (retorno_portfolio - taxa_livre_risco) / risco_portfolio
    return -indice_sharpe  # Negativo para maximização


# Função para calcular o Índice de Sharpe negativo
def negative_sharpe(weights, cov_matrix, returns, risk_free_rate=0.0005):
    # Retorno esperado do portfólio
    portfolio_return = np.dot(weights, returns)
    # Risco (volatilidade) do portfólio
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    # Índice de Sharpe negativo (minimização)
    return -(portfolio_return - risk_free_rate) / portfolio_stddev

# Número de ativos flexível com base no modelo ADL
num_ativos = len(retorno_esperado)
pesos_iniciais = np.array([1 / num_ativos] * num_ativos)

# Restrição: a soma dos pesos deve ser igual a 1
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
# Limites dos pesos (evitando posições negativas para diversificação limitada)
bounds = tuple((0.0, 1.0) for _ in range(num_ativos))

# Executa a otimização para maximizar o Índice de Sharpe
resultado = minimize(
    negative_sharpe,
    pesos_iniciais,
    args=(ultima_matriz_covariancia, retorno_esperado),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Resultados
pesos_ideais = resultado.x
sharpe_otimo = -resultado.fun

print("Pesos ótimos:", pesos_ideais)
print("Máximo Índice de Sharpe:", sharpe_otimo)


Pesos ótimos: [2.82045127e-10 0.00000000e+00 1.55280834e-08 0.00000000e+00
 9.98167491e-01 1.83249275e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.95305882e-10]
Máximo Índice de Sharpe: 0.5515751068767295
